## Spatial Entropy & Score Generator

This code generates the spatial entropy for all games, and the final score between the two teams. It saves this as a dataframe, with the idea being that we can then feed it into an LSTM algorithm.

In [11]:
# Import necessary libraries
import pandas as pd
import networkx as nx
import os
import matplotlib.pyplot as plt
import numpy as np
import math
import matplotlib.cm as mplcm
import matplotlib.colors as colors

In [12]:
## Import up sound alert dependencies
from IPython.display import Audio, display

def allDone():
    display(Audio(url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav', autoplay=True))
## Insert whatever audio file you want above

In [13]:
# Navigate to the relevant directory
os.chdir('/Users/Yuval/Desktop/Pawsey/data/')

In [14]:
cleaned_events = os.listdir('cleaned_events/')

In [15]:
def get_game(df, matchId):
    # Finds the match we're after.
    df = df.loc[df['matchId'] == matchId]
    return df

In [16]:
def continuous_time(df):
    # Converts to a continuous 90 min (ie 90*60 = 5400 second) game
    firsthalf = df.loc[df['matchPeriod'] == '1H']
    first_half_length = list(firsthalf['eventSec'])[-1]
    df.loc[ df.matchPeriod == '2H' , 'eventSec'] += first_half_length
    return df

In [17]:
def event_handling(df):
    # Cleans things up based on our interpretation of the events and how they relate to passes. Note the meaning of
    # the events:
    # 1 - duels (this is a hard one to define as it means many different things)
    # 2 - fouls
    # 3 - free kicks / corners (hence we add a node so that it picks these up as the first play in the chain)
    # 4 - keeper leaving their line
    # 7 - Touch, acceletation or clearance
    # 8 - Pass - simple/smart, cross (main data we're after for this)
    # 9 - Reflexes / save (however if tagged as inaccurate it means there was a goal)
    # 10- Shot (add a shot node)
    
    df = df.loc[ df['eventId'] != 9]
    df = df.loc[ df['eventId'] != 2]
    df = df.loc[ df['eventId'] != 4]
    df = df.loc[ (df['eventId'] != 1) | (df['Result'] == 'Accurate')]
    df = df.loc[ df['eventId'] != 7]
    
    #Check this later
    df = df.loc[df['attackMetres'] >= -30]

    # Create some new columns to see where passes ended up etc
    #df["next_event"] = df['eventId'].shift(-1)
    #df["next_teamId"] = df["teamId"].shift(-1)
    #df["next_player"] = df["shortName"].shift(-1)
    #df.loc[ df["next_event"] == 3, 'next_player'] = 0.5
    #df.loc[ df["eventId"] == 10, 'next_player'] = 0
    
    return df

In [18]:
#Define the zones

def zones(event_df, start_or_end):

    conditions = [
            (((event_df['x'+ start_or_end] < 16.5)& (event_df['x'+ start_or_end] >= 0) & (event_df['y'+ start_or_end] >= 0)& (event_df['y'+ start_or_end] <= 25))),
            (((event_df['x'+ start_or_end] < 16.5)& (event_df['x'+ start_or_end] >= 0) & (event_df['y'+ start_or_end] > 25)& (event_df['y'+ start_or_end] <= 36))),
            (((event_df['x'+ start_or_end] < 16.5)& (event_df['x'+ start_or_end] >= 0) & (event_df['y'+ start_or_end] > 36)& (event_df['y'+ start_or_end] <= 64))),
            (((event_df['x'+ start_or_end] < 16.5)& (event_df['x'+ start_or_end] >= 0) & (event_df['y'+ start_or_end] > 64)& (event_df['y'+ start_or_end] <= 75))),
            (((event_df['x'+ start_or_end] < 16.5)& (event_df['x'+ start_or_end] >= 0) & (event_df['y'+ start_or_end] > 75)& (event_df['y'+ start_or_end] <= 100))),
        
            (((event_df['x'+ start_or_end] <= 33)& (event_df['x'+ start_or_end] >= 16.5) & (event_df['y'+ start_or_end] >= 0)& (event_df['y'+ start_or_end] <=  25))),
            (((event_df['x'+ start_or_end] <= 33)& (event_df['x'+ start_or_end] >= 16.5) & (event_df['y'+ start_or_end] > 25)& (event_df['y'+ start_or_end] <=  50))),
            (((event_df['x'+ start_or_end] <= 33)& (event_df['x'+ start_or_end] >= 16.5) & (event_df['y'+ start_or_end] > 50)& (event_df['y'+ start_or_end] <=  75))),
            (((event_df['x'+ start_or_end] <= 33)& (event_df['x'+ start_or_end] >= 16.5) & (event_df['y'+ start_or_end] > 75)& (event_df['y'+ start_or_end] <=  100))),
            (((event_df['x'+ start_or_end] <= 50)& (event_df['x'+ start_or_end] > 33) & (event_df['y'+ start_or_end] >= 0))& (event_df['y'+ start_or_end] <=  25)),
            (((event_df['x'+ start_or_end] <= 50)& (event_df['x'+ start_or_end] > 33) & (event_df['y'+ start_or_end] > 25)& (event_df['y'+ start_or_end] <=  50))),
            (((event_df['x'+ start_or_end] <= 50)& (event_df['x'+ start_or_end] > 33) & (event_df['y'+ start_or_end] > 50)& (event_df['y'+ start_or_end] <=  75))),
            (((event_df['x'+ start_or_end] <= 50)& (event_df['x'+ start_or_end] > 33) & (event_df['y'+ start_or_end] > 75)& (event_df['y'+ start_or_end] <=  100))),
            (((event_df['x'+ start_or_end] <= 67)& (event_df['x'+ start_or_end] > 50) & (event_df['y'+ start_or_end] >= 0)& (event_df['y'+ start_or_end] <= 25))),
            (((event_df['x'+ start_or_end] <= 67)& (event_df['x'+ start_or_end] > 50) & (event_df['y'+ start_or_end] > 25)& (event_df['y'+ start_or_end] <=  50))),
            (((event_df['x'+ start_or_end] <= 67)& (event_df['x'+ start_or_end] > 50) & (event_df['y'+ start_or_end] > 50)& (event_df['y'+ start_or_end] <= 75))),
            (((event_df['x'+ start_or_end] <= 67)& (event_df['x'+ start_or_end] > 50) & (event_df['y'+ start_or_end] > 75)& (event_df['y'+ start_or_end] <= 100))),
            (((event_df['x'+ start_or_end] < 83.5)& (event_df['x'+ start_or_end] > 67) & (event_df['y'+ start_or_end] >= 0)& (event_df['y'+ start_or_end] <= 25))),
            (((event_df['x'+ start_or_end] < 83.5)& (event_df['x'+ start_or_end] > 67) & (event_df['y'+ start_or_end] > 25)& (event_df['y'+ start_or_end] <= 50))),
            (((event_df['x'+ start_or_end] < 83.5)& (event_df['x'+ start_or_end] > 67) & (event_df['y'+ start_or_end] > 50)& (event_df['y'+ start_or_end] <= 75))),
            (((event_df['x'+ start_or_end] < 83.5)& (event_df['x'+ start_or_end] > 67) & (event_df['y'+ start_or_end] > 75)& (event_df['y'+ start_or_end] <= 100))),
        
            (((event_df['x'+ start_or_end] <= 100) & (event_df['x'+ start_or_end] > 83.5) & (event_df['y'+ start_or_end] >= 0) & (event_df['y'+ start_or_end] <= 25))),
            (((event_df['x'+ start_or_end] <= 100) & (event_df['x'+ start_or_end] > 83.5) & (event_df['y'+ start_or_end] > 25) & (event_df['y'+ start_or_end] <=  36))),
            (((event_df['x'+ start_or_end] <= 100) & (event_df['x'+ start_or_end] > 83.5) & (event_df['y'+ start_or_end] > 36) & (event_df['y'+ start_or_end] <= 64))),
            (((event_df['x'+ start_or_end] <= 100) & (event_df['x'+ start_or_end] > 83.5) & (event_df['y'+ start_or_end] > 64) & (event_df['y'+ start_or_end] <= 75))),
            (((event_df['x'+ start_or_end] <= 100) & (event_df['x'+ start_or_end] > 83.5) & (event_df['y'+ start_or_end] > 75) & (event_df['y'+ start_or_end] <= 100)))
            ]

    # create a list of the values we want to assign for each condition
    values = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26]

    # create a new column and us np.select to assign values to it using our lists as arguments
    event_df['Zone'+ start_or_end] = np.select(conditions, values)
    
    return event_df


In [19]:
def transitions(df, teamIds):
    
    # Create a Transitions column which has the start and end zones
    df['Transition'] = list(zip(df['ZoneStart'], df['ZoneEnd']))
    
    # Get a transition list for each team
    trans = []
    for team in teamIds:    
        temp = df.loc[df['teamId'] == team]
        trans.append(list(temp['Transition']))

    return trans[0], trans[1]

In [20]:
def weighted_passes(passes):
    # Builds a network for the team based on the passes as edges
    n = nx.DiGraph()
    for (i,j) in passes:
        if n.has_edge(i,j):
            n[i][j]['weight'] += 1
        else:
            n.add_edge(i,j, weight = 1)
    
    # Remove node 0.5 as only used to signify start of plays from free kicks / corners
    if 0.5 in n.nodes():
        n.remove_node(0.5)
        
    return n

## Entropy

In [21]:
# Defines a function to calculate the entropy of each node from the transition matrix.
def entropy(row):
    enode = 0
    for item in row:
        if item > 0:
            enode -= item * math.log10(item) # Entropy formula for each node.
    return enode


In [22]:
def find_entropy(net):
    # This function attempts to calculate a team's entropy- i.e. their unpredictability.
    
    n = net
    
    # A stochastic graph gives the probabilities of connections between certain platers       
    prob = nx.stochastic_graph(n)
    
    # Allows us to sort nodes alphabetically so that we can easily remove shot node later
    playerlist = sorted(list(n.nodes()))
    
    # Generates transition matrix
    probmat = nx.adjacency_matrix(prob, nodelist = playerlist)
    probmat = probmat.todense()
    matrix = np.array(probmat) # turns it into numpy array so that we can operate on it

    entnode =  np.apply_along_axis( entropy, axis=1, arr = matrix ) # calculates entropy for each node

    entrop = sum(entnode)/len(n.nodes()) # Gets the average
        
    return entrop

In [23]:
def sliding_window(df, teams):
    # This function iterates sliding 10 minute windows, moving forward by 30 seconds each time and calculate a 
    # team's entropy during that 30 second window

    metrics1 = []
    metrics2 = []
    
    # Iterate sliding windows over the game length
    for i in range(0,int(list(df['eventSec'])[-1])-600,30):
        
        # locate the window in the database
        temp_df = df.loc[df['eventSec'] < i + 600]
        temp_df = temp_df.loc[temp_df['eventSec'] > i]
        
        # Get pass list, create a network, and find that network's entropy
        trans1, trans2 = transitions(temp_df,teams)

        net1 = weighted_passes(trans1)
        net2 = weighted_passes(trans2)

        # The team's total amount of passes during that 10 minute window 
        tot1 = 0
        tot2 = 0
        for item in list(net1.edges(data=True)):
            tot1 += item[2]['weight']
        for item in list(net2.edges(data=True)):
            tot2 += item[2]['weight']
        
        
        if tot1 > 0:
            met1 = find_entropy(net1)
        else:
            met1 = 0
        if tot2 > 0:
            met2 = find_entropy(net2)
        else:
            met2 = 0
            
        metrics1.append(met1)
        metrics2.append(met2)
    
    return metrics1, metrics2

In [34]:
def final_score(events_df, teams_df, matches_df):
    
    teams_ingame = sorted(list(set(events_df['teamId'])))
    matchid = list(events_df['matchId'])[0]
    
    name1 = list(teams_df.loc[teams_df['teamId'] == teams_ingame[0]]['name'])[0]
    name2 = list(teams_df.loc[teams_df['teamId'] == teams_ingame[1]]['name'])[0]
    
    mat = matches_df.loc[matches_df['wyId'] == matchid]
    label = list(mat['label'])[0]
    label = label.split(',')
    label = [i.split(' - ') for i in label]
    try:
        score = [[i.strip() for i in label[0]],[int(i.strip()) for i in label[1]]]
    except:
        score = [[i.strip() for i in label[0]],[i.strip() for i in label[1]]]
    
    #score is in the form [[teamName1, teamName2], [score1, score2]]
    
    return score

In [36]:
def game_iterator(master_df,league, teams_df, matches_df, teamid_dict):

    all_matches = list(set(master_df['matchId']))
    
    results = []

    for m in range(len(all_matches)):
        print(m)

        match = all_matches[m]

        df = get_game(master_df,match)
        df = continuous_time(df)
        df = event_handling(df)
        df = zones(df, "Start")
        df = zones(df, "End")

        teams = sorted(list(set(df['teamId'])))

        met1, met2 = sliding_window(df,teams)

        sco = final_score(df, teams_df, matches_df)
                   
        #sco is in the form [[teamName1, teamName2], [score1, score2]]
        score = []
        # Just a check to make sure entropy and score results will be callibrated to same team.
        if teamid_dict[sco[0][0]] == teams[0]:
            score = sco[1]
        else:
            score = [sco[1][1], sco[1][0]]
            
        results.append([[teams[0], teams[1], match], met1, met2, score])
    
    return results

In [37]:
# Read in teams df
Teams = pd.read_json('teams.json')
Teams = Teams.rename(columns = {'wyId': 'teamId'})

In [38]:
def main(name, Teams):
    
    df = pd.read_json('cleaned_events/'+name)
    
    Matches = pd.read_json('matches/matches_'+country[15:])
    
    teamid_dict = pd.Series(Teams.teamId.values, Teams.name.values).to_dict()
    official = pd.Series(Teams.teamId.values, Teams.officialName.values).to_dict()
    teamid_dict.update(official)
    
    results = game_iterator(df, name, Teams, Matches, teamid_dict)
        
    allDone()
    
    return results

In [39]:
onlyleagues = cleaned_events[:3]+cleaned_events[-2:]

In [93]:
allresults = []
for country in onlyleagues:
    print(country)
    r = main(country,Teams)
    allresults.append( r )
allDone() 

cleaned_events_England.json
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
27

cleaned_events_France.json
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270

cleaned_events_Spain.json
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270


cleaned_events_Germany.json
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
27

cleaned_events_Italy.json
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270


In [100]:
import json

for i in range(len(allresults)):
    # Open the file for writing
    with open('spa_ent_and_score/'+onlyleagues[i][15:], 'w') as F:
        # Use the json dumps method to write the list to disk
        F.write(json.dumps(allresults[i]))
    F.close()

## Same for Cups
Modify the above code slightly to give the spatial entropy and result for the cup games- but only for 90 mins, as will be too complex with Extra Time included.

In [40]:
onlycups = cleaned_events[3:-2]

In [41]:
onlycups

['cleaned_events_European_Championship.json', 'cleaned_events_World_Cup.json']

In [42]:
df= pd.read_json('cleaned_events/'+onlycups[0])

In [69]:
def cups_score(Matches,results):
    
    for item in results:
        match = item[0][2]
        
        Matches_temp = Matches.loc[Matches['wyId'] == match]
        
        label = list(Matches_temp['teamsData'])[0]
        
        teams = list(label.keys())
        
        sco1 = label[list(label.keys())[0]]['score']
        sco2 = label[list(label.keys())[0]]['score']
        
        if item[0][0] == int(teams[0]):
            item[3] = [sco1,sco2]
        elif item[0][1] == int(teams[0]):
            item[3] = [sco2, sco1]
        else:
            print('No match')
    
    return results

In [76]:
allresults = []
for cup in onlycups:
    print(cup)
    df = pd.read_json('cleaned_events/'+cup)
    df = df.loc[df['matchPeriod'] != 'E1']
    df = df.loc[df['matchPeriod'] != 'E2']
    df = df.loc[df['matchPeriod'] != 'P']
    
    Matches = pd.read_json('matches/matches_'+cup[15:])
    
    teamid_dict = pd.Series(Teams.teamId.values, Teams.name.values).to_dict()
    official = pd.Series(Teams.teamId.values, Teams.officialName.values).to_dict()
    teamid_dict.update(official)
    
    results = game_iterator(df, cup, Teams, Matches, teamid_dict)
    
    results = cups_score(Matches, results)
    
    allresults.append(results)
allDone() 

cleaned_events_European_Championship.json
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
cleaned_events_World_Cup.json
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63


In [77]:
minn = 500
for i in allresults:
    for j in i:
        if len(j[1]) == 0:
            print(j)
        if len(j[1]) < minn:
            minn = len(j[1])
minn

163

In [78]:
import json

for i in range(len(allresults)):
    # Open the file for writing
    with open('spa_ent_and_score/'+onlycups[i][15:], 'w') as F:
        # Use the json dumps method to write the list to disk
        F.write(json.dumps(allresults[i]))
    F.close()

In [73]:
allresults[0][0][1]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.3701577005432561,
 0.3873282324177822,
 0.4115440092030585,
 0.4132050209117704,
 0.43418941530014554,
 0.45706504349659227,
 0.44745900658070925,
 0.4171511683274571,
 0.42174943207779125,
 0.4251095646855811,
 0.41214221777603316,
 0.37269513763038253,
 0.3645286444898357,
 0.368933645081111,
 0.3735822769486368,
 0.36088699935244284,
 0.3455229252477905,
 0.3455229252477905,
 0.3325418922342024,
 0.31119093793464875,
 0.31544532785456747,
 0.3231333165734881,
 0.3069240599265224,
 0.30881455090928833,
 0.3108568140285629,
 0.2573928503926971,
 0.2841549633233122,
 0.29671078581593485,
 0.3015241111809772,
 0.2828623247680947,
 0.2956161414582262,
 0.3274429183746463,
 0.33209815648702873,
 0.33356165331205934,
 0.31072306944119704,
 0.32937561558448236,
 0.335069977120589,
 0.3542053984210909,
 0.3513366094706432,
 0.3716502047282302,
 0.38109219257279414,
 0.37761019916737065,
 0.37761019916737065,
 0.3777499873987128,
 0.374107204